## Imports

In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.cluster import KMeans

## Load images and masks

In [2]:
images_path = '/home/sofia/Documents/FING/Proyecto/clasificacion_de_nucleos/Lady/patches'
masks_filtered_path = '/home/sofia/Documents/FING/Proyecto/clasificacion_de_nucleos/Lady/masks_filtered'
images_name = os.listdir(images_path)

# Load images and masks
images = []
masks = []
for image_name in images_name[:100]:
    image = cv2.imread(os.path.join(images_path, image_name))
    mask = cv2.imread(os.path.join(masks_filtered_path, image_name),  cv2.IMREAD_GRAYSCALE)
    images.append(image)
    masks.append(mask)

# mask: channel 0 is background, channel 1 is for glands, channel 2 is for nuclei

## Calculate features for each nucleus of every image

In [3]:
from scipy.stats import entropy

def R(intensity_levels):
    norm_var = intensity_levels.var()/(255**2)
    return 1-1/(1+norm_var)

def U(p_intensity):
    return np.sum(p_intensity[0]**2)

def E(p_intensity):
    return entropy(p_intensity[0], base=2)    
    

In [4]:
nuclei_data = {}

'''
cv2.findContours parameters:
    mode -> cv2.RETR_EXTERNAL: retrieves only the extreme outer contours.
    method -> cv2.CHAIN_APPROX_SIMPLE: compresses horizontal, vertical, and diagonal segments and leaves only their end points.
'''

for i in tqdm(range(len(images))):

    # 1. Separate all nuclei instances in an image and calculate statistics.
    num_labels, nuclei_instances, stats, centroids = cv2.connectedComponentsWithStats(masks[i])

    contours = []
    perimeters = []
    areas = []
    circularity = []
    intensity = []
    texture_R = []
    texture_U = []
    texture_E = []
    gray_image = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)

    # 2. Iterate over each nucleus of an image and calculate features.
    for j in range(1,num_labels):

        nuclei_mask = ((nuclei_instances==j)*255).astype('uint8')
        output = cv2.findContours(image=nuclei_mask, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
        perimeter = cv2.arcLength(output[0][0],1)
        area = cv2.contourArea(output[0][0])
        contours.append(output[0][0])
        perimeters.append(perimeter)
        areas.append(area)
        intensity_levels = gray_image[nuclei_instances==j]
        intensity.append(intensity_levels.mean())
        if (perimeter**2!=0):
            circularity.append((np.pi*4*area)/(perimeter**2))
        else:
            # append circularity value of -1 for nuclei with perimeter^2=0
            circularity.append(-1)


        # 3. Calculate the probability density function of the intensity levels in the nucleus in order to calculate different metrics 
        # regarding nucleus texture
        p_intensity = np.histogram(intensity_levels, bins=np.arange(intensity_levels.min(), intensity_levels.max()+2), density=True)
        texture_R.append(R(intensity_levels))
        texture_U.append(U(p_intensity))
        texture_E.append(E(p_intensity))

    # 4. Save data of all nuclei in an image in a dictionary.
    nuclei_data[str(i)]={'contours':contours, 'perimeters': perimeters, 'areas':areas, 'circularity':circularity, 
                                       'intensity':intensity, 'texture_R': texture_R, 'texture_U': texture_U, 'texture_E': texture_E}


100%|█████████████████████████████████████████| 100/100 [00:49<00:00,  2.02it/s]


In [5]:
area_list = []
circularity_list = []
intensity_list = []
texture_R_list = []
texture_U_list = []
texture_E_list = []

for key in nuclei_data.keys():
    area_list += nuclei_data[key]['areas']
    circularity_list += nuclei_data[key]['circularity']
    intensity_list += nuclei_data[key]['intensity']
    texture_R_list += nuclei_data[key]['texture_R']
    texture_U_list += nuclei_data[key]['texture_U']
    texture_E_list += nuclei_data[key]['texture_E']

X = np.stack((area_list, circularity_list, intensity_list, texture_R_list, texture_U_list, texture_E_list), axis=1)
print (X.shape)

(23202, 6)


## Clustering algorithm

In [16]:
kmeans = KMeans(n_clusters=15, random_state=0, n_init="auto").fit(X)

## Inference over an example image

In [17]:
values, counts = np.unique(kmeans.labels_, return_counts=True)
print(values, counts)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14] [2217 1894  477 2496  120  957 2809  262 2124   34 1509 2272 2415 2931
  685]


In [18]:
generate_tuples = lambda N, R: [(random.randint(0, R), random.randint(0, R), random.randint(0, R)) for _ in range(N)]
colors = generate_tuples(len(values), 255)
colors

[(108, 13, 229),
 (0, 113, 214),
 (114, 61, 109),
 (195, 220, 239),
 (244, 210, 24),
 (197, 54, 210),
 (253, 49, 41),
 (132, 120, 217),
 (95, 118, 205),
 (203, 250, 207),
 (184, 213, 10),
 (123, 254, 94),
 (38, 39, 59),
 (149, 37, 181),
 (61, 82, 108)]

In [19]:
select_random_image = np.random.randint(0,len(images)-1)

selected_image_copy = images[select_random_image].copy()

'''
cv2.drawContours parameters:
    img -> source image
    contours -> contours which should be passed as a Python list
    index of contours -> useful when drawing individual contour. To draw all contours, pass -1.
    color
    thickness
'''

X_test = np.column_stack([nuclei_data[str(select_random_image)]['areas'], nuclei_data[str(select_random_image)]['circularity'], \
                             nuclei_data[str(select_random_image)]['intensity'], nuclei_data[str(select_random_image)]['texture_R'], \
                            nuclei_data[str(select_random_image)]['texture_U'],nuclei_data[str(select_random_image)]['texture_E']])
prediction = kmeans.predict(X_test)

for label in np.unique(kmeans.labels_):
    nuclei_idx = [idx[0] for idx in np.argwhere(prediction==label)]
    contours_list = []
    for idx in nuclei_idx:
        contours_list.append(nuclei_data[str(select_random_image)]['contours'][idx])

    cv2.drawContours(selected_image_copy, contours_list, -1, colors[label], 3)

# im_h = cv2.hconcat([images[select_random_image], selected_image_copy])

In [24]:
x,y,w,h = cv2.boundingRect(contours_list[0])
esto = cv2.rectangle(selected_image_copy,(x,y),(x+w,y+h),(0,255,0),2)


cv2.imshow('Contours', esto) 
cv2.waitKey(0) 
cv2.destroyAllWindows() 

In [38]:
colors[label]

array([ 70,  11, 148])

In [11]:
values, counts = np.unique(prediction, return_counts=True)
values, counts

(array([0, 1, 2, 3, 5, 6, 7, 8], dtype=int32),
 array([42, 28,  3, 42,  8, 35,  4, 23]))

In [12]:
prediction==0

array([False,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False,  True, False,  True, False, False,  True,  True, False,
       False,  True,  True, False, False,  True, False,  True, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False,  True, False, False,  True, False, False,
       False,  True, False, False, False,  True, False,  True, False,
        True, False, False,  True,  True, False, False, False, False,
       False,  True, False, False,  True, False, False, False, False,
       False,  True, False, False,  True, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False,

In [22]:
# nuclei_data[str(0)]['contours'][0]
cv2.drawContours(images[0], nuclei_data[str(0)]['contours'][0], -1, (0,255,0), 2)

array([[[181, 169, 191],
        [187, 171, 198],
        [196, 174, 209],
        ...,
        [181, 112, 140],
        [182, 118, 138],
        [184, 124, 139]],

       [[191, 185, 204],
        [193, 181, 205],
        [196, 175, 208],
        ...,
        [181, 105, 133],
        [182, 110, 133],
        [184, 115, 135]],

       [[200, 197, 213],
        [198, 188, 208],
        [195, 174, 202],
        ...,
        [153,  86, 114],
        [154,  90, 117],
        [156,  94, 120]],

       ...,

       [[236, 238, 236],
        [236, 238, 236],
        [236, 238, 236],
        ...,
        [242, 240, 240],
        [242, 240, 240],
        [243, 240, 240]],

       [[235, 238, 235],
        [235, 238, 235],
        [235, 238, 235],
        ...,
        [242, 240, 240],
        [242, 240, 240],
        [243, 240, 240]],

       [[234, 237, 234],
        [234, 237, 234],
        [234, 237, 234],
        ...,
        [242, 240, 240],
        [242, 240, 240],
        [243, 240, 240]]

In [25]:
selected_image = np.random.randint(0, len(images))

In [ ]:
    plt.imshow(cv2.addWeighted(cv2.cvtColor(images[random_images[i]], cv2.COLOR_BGR2RGB),0.7,masks[random_images[i]],0.3,0))
    plt.axis('off')

selected_image = np.random.randint(0, len(images), size=1)
print ('The selected image is : ', selected_image)

X_test = np.column_stack([nuclei_data[str(selected_image)]['areas'], nuclei_data[str(selected_image)]['circularity'], \
                             nuclei_data[str(selected_image)]['intensity'], nuclei_data[str(selected_image)]['texture_R'], \
                            nuclei_data[str(selected_image)]['texture_U'],nuclei_data[str(selected_image)]['texture_E']])
prediction = kmeans.predict(X_test)



for i in range(len(random_images)):

    fig.add_subplot(3, 3, i+1)
    
    
    nuclei_classification_mask = np.zeros(images[random_images[i]].shape)
    for j in range(np.unique(prediction)):

        prediction == j 
        


